In [11]:
# Load dataset
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras

def load_images(
    folder_name,
    images:list,
    labels:list,
    label:str
    ):
    
    folder_path = os.path.join(os.getcwd(), folder_name)

    for name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, name)

        if os.path.isfile(file_path):
            try:
                image = cv2.cvtColor(cv2.imread(file_path), cv2.COLOR_BGR2RGB)
                # Append several versions of the image
                images.append(image)
                labels.append(label)

                images.append(tf.image.flip_up_down(image))
                labels.append(label)

                images.append(tf.image.flip_left_right(image))
                labels.append(label)

                images.append(tf.image.random_contrast(image, 0.2, 0.7, seed=42))
                labels.append(label)
            except Exception as e:
                print(f'Error processing {file_path}: {e}')

images = []
labels = []

# Load images
load_images('cats-dataset/cat/resized', images, labels, 'cat')
load_images('dogs-dataset/dog/resized', images, labels, 'dog')

# Convert to numpy array and shuffle it
np.random.seed(42)
indices = np.random.permutation(len(images))
images = np.array(images)[indices]
labels = np.array(labels)[indices]

# Split into train and test samples

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
labels = encoder.fit_transform(labels)

X_train, X_test, y_train, y_test = train_test_split(
    images,
    labels,
    test_size=20,
    random_state=42,
    shuffle=True,
    stratify=labels
    )

print(y_test)

[0 0 0 1 0 0 1 1 1 1 1 0 0 1 1 0 0 1 0 1]


# Plot images with true and predicted labels

In [13]:
import matplotlib.pyplot as plt

def plot_images(
    images:np.array,
    lables:np.array, 
    predicted_labels:np.array,
    limit:int = 5
    )->None:
    fig, ax = plt.subplots(limit, 1, figsize=(15, 15))

    for index, image in enumerate(images):
        if index < limit:
            ax[index].imshow(image)
            ax[index].set_title(f'True label is {lables[index]}\nPredicted label is {predicted_labels[index]}')
            ax[index].axis('off')

    plt.tight_layout()
    plt.show()


# Plot metrics

In [14]:
def plot_metrics(
    metrics:list,
    scores:list,
    title:str | None = None
    )->None:

    fig, ax = plt.subplots(figsize=(10, 5))
    ax.bar(metrics, scores, width=0.2)
    ax.set_ylabel('Scores')
    ax.set_xlabel('Metrics')

    for i, s in enumerate(scores):
        ax.text(metrics[i], s, str(round(s, 3)), ha='center', va='bottom')

    fig.autofmt_xdate()
    plt.tight_layout()
    if title:
        plt.title(title) 
    plt.show()

# Building a CNN

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Normalize
X_train = X_train / 255.0
X_test = X_test / 255.0

# Build CNN model
model = keras.Sequential([
    # keras.layers.InputLayer(shape=(1, 240, 240, 3), batch_size=32),
    keras.layers.Conv2D(8, 4, activation='relu', strides=(2,1), padding='same', input_shape=(240, 240, 3)),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(16, 2, strides=(2, 1), padding='same', activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(32, 2, strides=(2,2), padding='same', activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(64, 3, strides=(2,2), padding='same', activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    # keras.layers.Conv2D(2, 3, strides=(2,2), padding='same', activation='relu'),
    # keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Dropout(0.3),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(264, activation='relu'),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(2, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Train
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
)

# Evaluate
test_loss, test_accuracy = model.evaluate(X_test, y_test)
train_loss, train_accuracy = model.evaluate(X_train, y_train)
plot_metrics(
    ['test', 'train'],
    [test_accuracy, train_accuracy],
    "Model's accuracy on train and test samples"
)

# Predictions
y_pred = model.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)

# Metrics
accuracy = accuracy_score(y_test, y_pred_classes)
f1 = f1_score(y_test, y_pred_classes, average='weighted')
precision = precision_score(y_test, y_pred_classes, average='weighted')
recall = recall_score(y_test, y_pred_classes, average='weighted')

plot_metrics(
    ['accuracy', 'precision', 'recall', 'f1'],
    [accuracy, precision, recall, f1],
    "Model's metrics"
)

plot_images(
    X_test,
    encoder.inverse_transform(y_test),
    encoder.inverse_transform(y_pred_classes),
    limit=10
)

/home/nykfer/projects/nn/.venv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-11-21 20:25:55.676183: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 373248000 exceeds 10% of free system memory.


Epoch 1/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 16s 434ms/step - accuracy: 0.5259 - loss: 0.6935 - val_accuracy: 0.5882 - val_loss: 0.6873
Epoch 2/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 8s 458ms/step - accuracy: 0.5611 - loss: 0.6833 - val_accuracy: 0.5294 - val_loss: 0.6757
Epoch 3/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 4s 254ms/step - accuracy: 0.6204 - loss: 0.6518 - val_accuracy: 0.6250 - val_loss: 0.6534
Epoch 4/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 282ms/step - accuracy: 0.6537 - loss: 0.6367 - val_accuracy: 0.6250 - val_loss: 0.6443
Epoch 5/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 254ms/step - accuracy: 0.6333 - loss: 0.6326 - val_accuracy: 0.5515 - val_loss: 0.6745
Epoch 6/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 4s 248ms/step - accuracy: 0.6185 - loss: 0.6263 - val_accuracy: 0.6544 - val_loss: 0.6130
Epoch 7/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 8s 459ms/step - accuracy: 0.7000 - loss: 0.5611 - val_accuracy: 0.7059 - val_loss: 0.5989
Epoch 8/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 6s 366ms/step - accuracy: 0.7481 - loss: 0.5101 - val_accuracy: 0